nanoGPT
========
Following a tutorial from [Andrej Karpathy](https://karpathy.ai/):
- [Let's build GPT: from scratch, in code, spelled out.](https://youtu.be/kCc8FmEb1nY?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ)

## Setup

In [2]:
# download the dataset file from github
# !wget https://github.com/karpathy/char-rnn/raw/master/data/tinyshakespeare/input.txt

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import matplotlib.pyplot as plt
# %matplotlib inline

In [96]:
# hyperparameters
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

In [29]:
# read in all the text
text = open('input.txt', 'r', encoding='utf-8').read()
print(f'{len(text)=}\n---\n{text[:100]}')

len(text)=1115394
---
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [30]:
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars), vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [31]:
# encode the input into a computer friendly format
# here we are tokenizing character by character, in real life chatGPT uses a byte-pair encoding (BPE),
# it's a sub-word units, which means they don't use entire words as token, only chunks of binary

# in real life we'd use something like
# import tiktoken
# enc = tiktoken.get_encoding('gpt2')
# enc.encode('I for one welcome our new AI overlords')
# enc.decode([40, 329, 530, 7062, 674, 649, 9552, 12893, 3669])

# for our toy example we use some manual encoding
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# we have a good candidate for quickcheck-style auto generated tests here
s = 'string generated by quickcheck from our vocab'
assert s == decode(encode(s)), f'{s} did not encore/decode correctly'

print(encode('I for one welcome our new AI overlords'))
print(decode([21, 1, 44, 53, 56, 1, 53, 52, 43, 1, 61, 43, 50, 41, 53, 51, 43, 1, 53, 59, 56, 1, 52, 43, 61, 1, 13, 21, 1, 53, 60, 43, 56, 50, 53, 56, 42, 57]))

[21, 1, 44, 53, 56, 1, 53, 52, 43, 1, 61, 43, 50, 41, 53, 51, 43, 1, 53, 59, 56, 1, 52, 43, 61, 1, 13, 21, 1, 53, 60, 43, 56, 50, 53, 56, 42, 57]
I for one welcome our new AI overlords


In [32]:
# encode the dataset
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [33]:
# split the data into training set and validation set
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [92]:
# chunk the dataset into smaller pieces for training
# in practice we will train in all sizes < chunk
# e.g. for a chunk size of 8 we will train on chunks of size 1 to 8

def get_batch(dataset):
    data = train_data if dataset == 'train' else val_data
    ix = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')
print(xb[:2])
print(yb[:2])

tensor([[58, 58, 50, 43,  1, 39, 41, 41],
        [53, 59, 56,  1, 61, 47, 50, 50]], device='cuda:0')
tensor([[58, 50, 43,  1, 39, 41, 41, 59],
        [59, 56,  1, 61, 47, 50, 50, 12]], device='cuda:0')


In [93]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

## Implem: Bigram Model

In [94]:
# B: batch size, T: sequence length, C: channel is vocab size
torch.manual_seed(1337) # for reproducibility

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # <???> why is it `vocab_size * vocab_size` and not `vocab_size * embeddings_dim`?
        # I find it confusing for embeddings to have exactly `vocab_size` dimensions.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # idx and targets are (B, T)
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            # juggle with tensor shapes to match pytorch's cross_entropy
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)
logits, loss = m(xb, yb)
print(logits.shape)
# Note: by default we have a vocab of size 65, so before training we expect a loss of: -ln(1/65) = 4.17
print(loss)

torch.Size([256, 65])
tensor(4.7940, device='cuda:0', grad_fn=<NllLossBackward0>)


In [101]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [102]:
# train
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f'step {iter}: train loss {losses["train"]:.4f}, val loss {losses["val"]:.4f}')

    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.7288, val loss 4.7226
step 300: train loss 2.8099, val loss 2.8241
step 600: train loss 2.5475, val loss 2.5532
step 900: train loss 2.4989, val loss 2.5202
step 1200: train loss 2.4871, val loss 2.5082
step 1500: train loss 2.4793, val loss 2.4953
step 1800: train loss 2.4725, val loss 2.4870
step 2100: train loss 2.4704, val loss 2.4801
step 2400: train loss 2.4616, val loss 2.4873
step 2700: train loss 2.4609, val loss 2.4907


In [106]:
# sample from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=100)[0].tolist()))


Wawice my.
Whastacom oroup
Yowhthetof isth ble mil ndill, bes iree senghin lat Heriliovets, anend l 
